In [16]:
# !cat file.txt > file2.txt
# !pip install ...

In [19]:
import numpy as np
import pandas as pd

In [20]:
df = pd.read_csv('data/portfolio.csv')

In [22]:
df.head()

,name,shares,price
0,AA,100,32.20
1,IBM,50,91.10
2,CAT,150,83.44
3,MSFT,200,51.23
4,GE,95,40.37


In [24]:
df.tail(2)

,name,shares,price
5,MSFT,50,65.10
6,IBM,100,70.44


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   name    7 non-null      object 
 1   shares  7 non-null      int64  
 2   price   7 non-null      float64
dtypes: float64(1), int64(1), object(1)
memory usage: 296.0+ bytes


In [43]:
# df.name    #<-- Series
# df['name']    #<-- Series object
df[['name']]  #<--  dataframe
arr =df[['price']].values  #<-- #numpy array
df['price2']=np.roll(arr,2)
df

,name,shares,price,price2
0,AA,100,32.20,65.10
1,IBM,50,91.10,70.44
2,CAT,150,83.44,32.20
3,MSFT,200,51.23,91.10
4,GE,95,40.37,83.44
5,MSFT,50,65.10,51.23
6,IBM,100,70.44,40.37


In [47]:
# map  
df.apply(lambda x: x['name'].upper(), axis= 'columns') # str 

0      AA
1     IBM
2     CAT
3    MSFT
4      GE
5    MSFT
6     IBM
dtype: object

In [63]:
a= df['name'].values.astype(str)

df['name']= np.char.lower(a)

In [71]:
# df.set_index(df.name,inplace=True)
df.index= df.name

In [74]:
df.drop('name', inplace=True, axis = 1)

In [80]:
df.reset_index(inplace= True)
df.drop('price2', inplace = True, )

ValueError: cannot insert level_0, already exists

In [82]:
df.drop(['level_0','index'], inplace = True, axis= 1)

In [90]:
total = df.shares * df.price
df.insert(0,'New_total',total)

In [99]:
df['prv_month']= df['total'].shift(periods=1).fillna(value=0)

In [100]:
df

,New_total,name,shares,price,total,prv_month
0,3220.00,aa,100,32.20,3220.00,0.00
1,4555.00,ibm,50,91.10,4555.00,3220.00
2,12516.00,cat,150,83.44,12516.00,4555.00
3,10246.00,msft,200,51.23,10246.00,12516.00
4,3835.15,ge,95,40.37,3835.15,10246.00
5,3255.00,msft,50,65.10,3255.00,3835.15
6,7044.00,ibm,100,70.44,7044.00,3255.00


In [101]:
df['profit/loss'] = df.total - df.prv_month

In [105]:
# df.drop('prv_month', axis= 1, inplace=True)
df

,New_total,name,shares,price,total,profit/loss
0,3220.00,aa,100,32.20,3220.00,3220.00
1,4555.00,ibm,50,91.10,4555.00,1335.00
2,12516.00,cat,150,83.44,12516.00,7961.00
3,10246.00,msft,200,51.23,10246.00,-2270.00
4,3835.15,ge,95,40.37,3835.15,-6410.85
5,3255.00,msft,50,65.10,3255.00,-580.15
6,7044.00,ibm,100,70.44,7044.00,3789.00


In [177]:
# def ccy(x):
#     val = x.values[0]
#     if val < 0:
#         return f'$({abs(val):.2f})'
#     else:
#         return f'${val:.2f}'
    
def ccy(x):
    val = x.values[0]
    new_val = f'&&{val:.2f}' if val > 0 else f'&({abs(val):.2f})'
    return new_val
    
df2 = df.groupby('name')['profit/loss'].agg(['mean','max','min',ccy])

 

In [175]:
float('(222)')

ValueError: could not convert string to float: '(222)'

In [145]:
df2.ccy.cumsum()

name
aa                                            &&3220.00
cat                                  &&3220.00&&7961.00
ge                         &&3220.00&&7961.00&(6410.85)
ibm               &&3220.00&&7961.00&(6410.85)&&1335.00
msft    &&3220.00&&7961.00&(6410.85)&&1335.00&(2270.00)
Name: ccy, dtype: object

In [161]:
df2['ccy']= df2['ccy'].str.replace('&','').replace('()','')

In [158]:
pd.to_numeric(df2['ccy'])

ValueError: Unable to parse string "(6410.85)" at position 2

In [151]:
df2.ccy.cumsum()

name
aa                                      3220.00
cat                              3220.007961.00
ge                      3220.007961.00(6410.85)
ibm              3220.007961.00(6410.85)1335.00
msft    3220.007961.00(6410.85)1335.00(2270.00)
Name: ccy, dtype: object

In [152]:
df2.ccy.dtype

dtype('O')

In [176]:
df2['ccy']= df2['ccy'].str.extract(r'(\d+)')
df2['ccy']= df2['ccy'].astype(float)

In [183]:

# df2[['ccy']].replace(r'\W+',value='')

,ccy
name,
aa,&&3220.00
cat,&&7961.00
ge,&(6410.85)
ibm,&&1335.00
msft,&(2270.00)


In [190]:
# for i in df2.items():
#     print(i)

    
for i in df2.iteritems():
    print(i)

('mean', name
aa      3220.000
cat     7961.000
ge     -6410.850
ibm     2562.000
msft   -1425.075
Name: mean, dtype: float64)
('max', name
aa      3220.00
cat     7961.00
ge     -6410.85
ibm     3789.00
msft    -580.15
Name: max, dtype: float64)
('min', name
aa      3220.00
cat     7961.00
ge     -6410.85
ibm     1335.00
msft   -2270.00
Name: min, dtype: float64)
('ccy', name
aa       &&3220.00
cat      &&7961.00
ge      &(6410.85)
ibm      &&1335.00
msft    &(2270.00)
Name: ccy, dtype: object)


In [194]:
import sys
val =df2.iteritems()

print(sys.getsizeof(val))
print(sys.getsizeof(df2))

112
767


In [198]:
for i in df2.iterrows():
    print(i[1])
    break

mean       3220.0
max        3220.0
min        3220.0
ccy     &&3220.00
Name: aa, dtype: object


In [199]:
df2

,mean,max,min,ccy
name,,,,
aa,3220.000,3220.00,3220.00,&&3220.00
cat,7961.000,7961.00,7961.00,&&7961.00
ge,-6410.850,-6410.85,-6410.85,&(6410.85)
ibm,2562.000,3789.00,1335.00,&&1335.00
msft,-1425.075,-580.15,-2270.00,&(2270.00)


In [212]:
df2.loc[['aa','ge','msft'],['ccy','max','mean']] #row and the column the names
# df.iloc

,ccy,max,mean
name,,,
aa,&&3220.00,3220.00,3220.000
ge,&(6410.85),-6410.85,-6410.850
msft,&(2270.00),-580.15,-1425.075


In [225]:
df2= df2.loc[:,'max':]
df2.reset_index(inplace=True)

df2.loc[2:,:]
df2.loc[2:]



,index,max,mean
2,2,-6410.85,-6410.850
3,3,3789.00,2562.000
4,4,-580.15,-1425.075


In [218]:
df2.iloc[0:2,1:]

,max,mean
name,,
aa,3220.0,3220.0
cat,7961.0,7961.0


In [ ]:
<tag attributes> content </tag>

<div> </div>
<span>
<a>
<h1-6>
<input>
<img>
<video>
<form>

class
id
href== url links are stored


In [ ]:
<div class='something' id='two'>
 description
</div>
<div class='something' id= 'one'>
price amount
</div>
<div class='something'>
picture
</div>
<div class='something'>
...
</div>





<style>
.something{
    color: red
}

'#' one{
    color: blue
}


</style>


In [232]:
arr =np.array([1,2,3,4])
np.repeat(arr,[1,0])

ValueError: operands could not be broadcast together with shape (4,) (2,)